# Online Training

A variation of SBI is Sequential Neural Posterior Estimation (SNPE), where the model is trained online, i.e., in multiple rounds. In each round, simulations are generated from the current posterior estimate, and the model is updated with these new simulations. This approach can be more efficient in terms of simulation budget, especially when the prior is broad and the posterior is narrow.

However, the model is no longer amortized, as it is specialized to the specific observation after training. This means that a new model must be trained for each new observation, which can be computationally expensive if many observations need to be analyzed.

In [1]:
from synference import SBI_Fitter

fitter = SBI_Fitter.init_from_hdf5(model_name="test", 
                                   hdf5_path="../example_grids/test_model_grid.hdf5")

fitter.create_feature_array()

2025-11-04 16:16:18,905 | synference | INFO     | ---------------------------------------------
2025-11-04 16:16:18,906 | synference | INFO     | Features: 8 features over 10000 samples
2025-11-04 16:16:18,907 | synference | INFO     | ---------------------------------------------
2025-11-04 16:16:18,907 | synference | INFO     | Feature: Min - Max
2025-11-04 16:16:18,907 | synference | INFO     | ---------------------------------------------
2025-11-04 16:16:18,908 | synference | INFO     | JWST/NIRCam.F070W: 3.642582 - 43.046 AB
2025-11-04 16:16:18,908 | synference | INFO     | JWST/NIRCam.F090W: 3.246181 - 40.840 AB
2025-11-04 16:16:18,908 | synference | INFO     | JWST/NIRCam.F115W: 2.951415 - 38.513 AB
2025-11-04 16:16:18,908 | synference | INFO     | JWST/NIRCam.F150W: 2.685013 - 37.131 AB
2025-11-04 16:16:18,909 | synference | INFO     | JWST/NIRCam.F200W: 2.763217 - 35.357 AB
2025-11-04 16:16:18,909 | synference | INFO     | JWST/NIRCam.F277W: 3.175404 - 33.451 AB
2025-11-04 16

(array([[36.56076291, 26.84319806, 27.53516844, ..., 29.23849269,
         31.91549615, 21.2721818 ],
        [35.52377159, 26.00320115, 26.54587496, ..., 27.49418495,
         30.62425058, 20.28062427],
        [34.67610464, 24.86377695, 26.13228615, ..., 25.86075064,
         29.02157932, 19.71605108],
        ...,
        [31.50460699, 23.11560968, 25.21971448, ..., 23.47749148,
         24.17974863, 18.65903756],
        [31.02450546, 23.00129839, 25.14247733, ..., 23.05211449,
         23.43121362, 18.55571793],
        [30.72834507, 22.91391853, 25.42996732, ..., 22.86743574,
         22.91672148, 18.80079228]]),
 ['JWST/NIRCam.F070W',
  'JWST/NIRCam.F090W',
  'JWST/NIRCam.F115W',
  'JWST/NIRCam.F150W',
  'JWST/NIRCam.F200W',
  'JWST/NIRCam.F277W',
  'JWST/NIRCam.F356W',
  'JWST/NIRCam.F444W'])

Now we will recreate the simulator from the grid data stored in the HDF5 file.

In [2]:
fitter.recreate_simulator_from_grid()

ValueError: Dust model AttenuatedEmission not found in <module 'synthesizer.emission_models.attenuation' from '/Users/user/Documents/PhD/synthesizer/synthesizer/src/synthesizer/emission_models/attenuation/__init__.py'>. Cannot create GalaxySimulator.

Now we can choose an observation for our multiple rounds of online training. Here, we will randomly select one of the simulations from our grid as the observation.

In [ ]:
index = 20
sample = fitter.feature_array[index]
true_params = fitter.fitted_parameter_array[index]

Now we can run our online SBI model - to do this we set 'learning_type' to 'online', specify the number of online rounds with 'num_online_rounds', and provide our chosen observation with 'online_training_xobs'. We also set the number of simulations per round with 'num_simulations'. The engine is set to 'SNPE' to use Sequential Neural Posterior Estimation, but SNLE and SNRE are also available for online training.

In [ ]:
fitter.run_single_sbi(
    online_training_xobs=sample,
    learning_type="online",
    engine="SNPE",
    num_simulations=10_000,
    num_online_rounds=3,
)